# Логистическая регрессия: Южный Парк

Импортирую всякие модули

In [2]:
import pandas
import numpy as np
import matplotlib.pyplot as plt

from nltk import word_tokenize 
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
from sklearn.linear_model import LogisticRegression

Вытаскиваю и нормирую данные.

In [3]:
SO_PARK = pandas.read_csv('All-seasons.csv', sep=',')
SO_PARK.head()

,Season,Episode,Character,Line
0,10,1,Stan,"You guys, you guys! Chef is going away. \n"
1,10,1,Kyle,Going away? For how long?\n
2,10,1,Stan,Forever.\n
3,10,1,Chef,I'm sorry boys.\n
4,10,1,Stan,"Chef said he's been bored, so he joining a gro..."


Под главными персонажами я поняла четвёрку главных.

In [4]:
stan = SO_PARK[SO_PARK['Character'] == 'Stan']
kyle = SO_PARK[SO_PARK['Character'] == 'Kyle']
eric = SO_PARK[SO_PARK['Character'] == 'Cartman']
kenny = SO_PARK[SO_PARK['Character'] == 'Kenny']

In [5]:
print('stan: ' + str(len(stan)))
print('kyle: ' + str(len(kyle)))
print('eric: ' + str(len(eric)))
print('kenny: ' + str(len(kenny)))

stan: 7680
kyle: 7099
eric: 9774
kenny: 881


Данные несбалансированы, потому что Кенни говорит слишком мало. При этом данных очень много, потому что это все сезоны. Поэтому я, пожалуй, просто выкину часть лишних данных.

In [9]:
less_stan = stan.sample(n=len(kenny))
less_kyle = kyle.sample(n=len(kenny))
less_eric = eric.sample(n=len(kenny))
balanced = pandas.concat([less_stan, less_kyle, less_eric, kenny])
balanced.groupby('Character').describe()

Episode        Line Season
Character                                  
Cartman   count      881         881    881
          unique      18         875     18
          top          4  Butters.\n      4
          freq        75           3     79
Kenny     count      881         881    881
          unique      18         754     18
          top          3   (Yeah!)\n      3
          freq       106          17    132
Kyle      count      881         881    881
          unique      18         843     18
          top          9     What?\n      2
          freq        79          13     97
Stan      count      881         881    881
          unique      18         853     18
          top         10     What?\n      2
          freq        75           8     95

Теперь датасет сбалансирован.

Дальше я попробую лемматизировать и не лемматизировать. Лемматизировать – потому что это кажется разумным решением, в конце концов, о человеке должны больше говорять лексемы, которые он употребляет, чем словоформы. Не лемматизировать, потому что в прошлом дз в задаче классификации писем внезапно лучше показали семя не лемматизированные словари.

In [39]:
# не лемматизируя
bow_non_lem = CountVectorizer()
bow_non_lem.fit_transform(balanced['Line'])
bowed_non_lem = bow_non_lem.transform(balanced['Line'])

In [40]:
# функция для лемматизации. коммент: я пыталась сделать это как полагается, и сначала сделать POS-tagging,
# но лемматизатор ворднета не идеален. он, например, вообще не умеет лемматизировать местоимения...

from nltk.stem.wordnet import WordNetLemmatizer
lmtzr = WordNetLemmatizer()

def lemmatize(word):
    noun = lmtzr.lemmatize(word)
    verb = lmtzr.lemmatize(word, 'v')
    if verb != word:
        return verb
    return noun

def tokenize_lemmatize(text):
    text = word_tokenize(text.lower())
    return [lemmatize(word) for word in text if word not in '!?,.":;']

bow_lem = CountVectorizer(tok)
bow_lem.fit_transform(balanced['Line'])
bowed_lem = bow_lem.transform(balanced['Line'])

In [37]:
print(bow_non_lem.vocabulary_)

{'weave': 3611, 'manager': 2022, 'fbi': 1174, 'scared': 2856, 'adverse': 77, 'imaginary': 1700, 'isn': 1745, 'essay': 1092, 'goods': 1407, 'perfect': 2458, 'biblical': 342, 'aclu': 55, 'ms': 2183, 'bushes': 475, 'swearing': 3224, 'spirituality': 3095, 'deal': 837, 'wrestling': 3740, 'ripped': 2784, 'jeez': 1761, 'earth': 1013, 'say': 2850, 'gross': 1440, 'these': 3322, 'field': 1198, 'gibson': 1366, 'sow': 3075, 'dollar': 940, 'murders': 2190, 'story': 3164, 'revenge': 2764, 'tcha': 3273, 'yuh': 3791, 'flashback': 1234, 'jonas': 1787, 'revamp': 2763, 'funny': 1318, 'rehab': 2723, 'rides': 2774, 'thinking': 3330, 'still': 3147, 'ladybug': 1868, 'visit': 3549, 'strong': 3177, 'tough': 3393, 'gnomes': 1387, 'planetarium': 2512, 'them': 3318, 'offs': 2321, 'giggling': 1369, 'hude': 1647, 'cousin': 759, 'clouds': 656, 'action': 58, 'tools': 3380, 'belonged': 328, 'smuggle': 3031, 'eighty': 1047, 'workin': 3720, 'sensual': 2906, 'don': 944, 'lazy': 1888, 'butters': 483, 'washes': 3591, 'such

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data[features], 
                                                    data['admit'], 
                                                    test_size=0.2)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
y_train.hist()